In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chisquare
import scipy.stats as stats

In [2]:
df = pd.read_csv("titanic.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df = df.drop("Cabin", axis=1)
df = df.drop("Ticket", axis=1)
df['Age'].fillna(df['Age'].mean(), inplace = True)

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [7]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
dtype: int64

## 1. Count the total number of passengers on board the Titanic (2 Mark)

In [8]:
df['PassengerId'].count()

891

## 2. Count the number of passengers who survived the sinking of the Titanic. (2 Mark)

In [9]:
np.count_nonzero(df['Survived'])

342

## 3. Measure the percentage of passengers who survived the sinking of the Titanic. (2 Mark)

In [10]:
s = df.Survived
counts = s.value_counts()
percent100 = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'Counts': counts, 'Percentage': percent100})

,Counts,Percentage
0,549,61.6%
1,342,38.4%


## 4. Count the number of passengers based on gender (2 Mark)

In [11]:
s = df.Sex
counts = s.value_counts()
pd.DataFrame({'Counts': counts})

,Counts
male,577
female,314


## 5. Run a Pearson’s Chi-squared test, without using Scipy function, to test the following hypothesis: (4 mark) 
## Hypothesis: The proportion of females onboard who survived the sinking of the Titanic was higher than the proportion of males onboard who survived the sinking of the Titanic.

In [12]:
table = pd.crosstab(columns=df.Sex,index=df.Survived, margins=True)
table.columns = ['Male','Female','Row_Total']
table.index = ['Dead','Survived','Col_Total']

In [13]:
table

,Male,Female,Row_Total
Dead,81,468,549
Survived,233,109,342
Col_Total,314,577,891


In [14]:
observed = table.iloc[0:2, 0:2]
observed

,Male,Female
Dead,81,468
Survived,233,109


In [15]:
expected = np.outer(table["Row_Total"][0:2],table.loc["Col_Total"][0:2]) / 1000
expected = pd.DataFrame(expected)
expected.columns = ["Male", "Female"]
expected.index = ["Dead", "Survived"]
expected

,Male,Female
Dead,172.386,316.773
Survived,107.388,197.334


In [16]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()
print(chi_squared_stat)

307.1117228626887


In [17]:
crit = stats.chi2.ppf(q = 0.95,df = 8)
print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,df=8)
print("P value")
print(p_value)

Critical value
15.50731305586545
P value
0.0


## 6. What is the inference based on the test? (1 mark)

In [19]:
if chi_squared_stat < crit:
    print("""At 0.95 level of significance, we reject the null hypotheses. They are not independent.""" )
else:
    print("""At 0.95 level of significance, we accept the null hypotheses. They are independent.""" )

At 0.95 level of significance, we accept the null hypotheses. They are independent.


## 7. Repeat questions 5 and 6 using Scipy function. (2 marks)

In [20]:
chisquare(table)

Power_divergenceResult(statistic=array([133.68471338, 311.7729636 , 258.81818182]), pvalue=array([9.34831489e-30, 1.99232943e-68, 6.28558799e-57]))

In [21]:
stats = stats.chi2_contingency(observed=observed)
# The returned data includes: chi_squared_stat, p_value, df, expected_crosstab
print(stats)

(260.71702016732104, 1.1973570627755645e-58, 1, array([[193.47474747, 355.52525253],
       [120.52525253, 221.47474747]]))


In [22]:
if stats[0] < stats[1]:
    print("""We reject the null hypotheses. They are not independent.""" )
else:
    print("""We accept the null hypotheses. They are independent.""" )

We accept the null hypotheses. They are independent.
